<a href="https://colab.research.google.com/github/saktiworkstation/road-to-ai-developer/blob/main/learn_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=3021b517098024bb3feaf5e5898c27f17bd61a69c5ab115e763f9a6a7be91242
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print(spark)
print(spark.version)

3.5.3


In [ ]:
df = spark.read.csv("/content/flights_small.csv", header=True, inferSchema=True)
df.write.saveAsTable("flights")

In [ ]:
print(spark.catalog.listTables())

[Table(name='flights', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]


In [ ]:
query = "FROM flights SELECT * LIMIT 10"
flights10 = spark.sql(query)
flights10.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
long_flights1 = flights10.filter("distance > 1000")
long_flights1.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+



In [ ]:
pd_long_flight1 = long_flights1.toPandas()
pd_long_flight1.to_csv('long_flight.csv')

In [ ]:
query = "SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest"


flight_counts = spark.sql(query)
pd_counts = flight_counts.toPandas()

print(pd_counts.head())

  origin dest    N
0    SEA  RNO    8
1    SEA  DTW   98
2    SEA  CLE    2
3    SEA  LAX  450
4    PDX  SEA  144


In [ ]:
pd_counts.to_csv('flight_counts.csv')

In [ ]:
flights = spark.table('flights')
flights = flights.withColumn('duration_hrs', flights.air_time / 60)
flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [ ]:
query = "SELECT origin, dest, air_time/60 as duration_hrs From flights"

flight_at = spark.sql(query)
flight_at.show()

+------+----+------------------+
|origin|dest|      duration_hrs|
+------+----+------------------+
|   SEA| LAX|               2.2|
|   SEA| HNL|               6.0|
|   SEA| SFO|              1.85|
|   PDX| SJC|1.3833333333333333|
|   SEA| BUR|2.1166666666666667|
|   PDX| DEN|2.0166666666666666|
|   PDX| OAK|               1.5|
|   SEA| SFO|1.6333333333333333|
|   SEA| SAN|              2.25|
|   SEA| ORD|               3.3|
|   SEA| LAX|2.1666666666666665|
|   SEA| PHX| 2.566666666666667|
|   SEA| LAS|2.1166666666666667|
|   SEA| ANC|              3.05|
|   SEA| SFO|              2.15|
|   PDX| SFO|               1.5|
|   SEA| SMF|1.2666666666666666|
|   SEA| MDW|               3.6|
|   SEA| BOS| 4.833333333333333|
|   PDX| BUR|              1.85|
+------+----+------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F

delayed_flights = flights10.filter(flights10.dep_delay > 30)

delay_analysis = delayed_flights.groupBy("month", "carrier") \
    .agg(F.avg("dep_delay").alias("avg_dep_delay")) \
    .orderBy("month", "carrier")

delay_analysis.show()

+-----+-------+-------------+
|month|carrier|avg_dep_delay|
+-----+-------+-------------+
|    4|     WN|         45.0|
|    7|     WN|         42.0|
+-----+-------+-------------+



#penjelasan


*   mengimpor modul functions dari PySpark dan mengalihkannya menjadi F untuk fungsi seperti avg, sum, count, dll.
*   Metode .agg() digunakan untuk melakukan agregasi pada kelompok data yang telah dikelompokkan.



In [ ]:
long_distance_flights = flights10.filter(flights10.distance > 500)

distance_analysis = long_distance_flights.groupBy("carrier") \
    .agg(F.sum("distance").alias("total_distance"),
         F.count("flight").alias("total_flights")) \
    .orderBy("carrier")

distance_analysis.show()

+-------+--------------+-------------+
|carrier|total_distance|total_flights|
+-------+--------------+-------------+
|     AS|          6385|            4|
|     VX|          2312|            3|
|     WN|          2103|            3|
+-------+--------------+-------------+

